In [1]:
import $ivy.`org.apache.spark:spark-sql_2.13:3.3.2`
import org.apache.spark.sql._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions._

val spark = SparkSession.builder()
  .appName("SparkTest")
  .master("local[*]")
  .getOrCreate()
spark.sparkContext.setLogLevel("ERROR")
  
// http://localhost:4040/jobs/
println("Spark is ready!")

SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties


Spark is ready!


import $ivy.$                                      

import org.apache.spark.sql._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions._


spark: SparkSession = org.apache.spark.sql.SparkSession@7ec1b4cb

In [2]:
import spark.implicits._

case class EmployeeDepartment(employee_id: Int, department_id: Int, primary_flag: String)

val employeeDepartment: Seq[(Int, Int, String)] = Seq(
  (1, 1, "N"),
  (2, 1, "Y"),
  (2, 2, "N"),
  (3, 3, "N"),
  (4, 2, "N"),
  (4, 3, "Y"),
  (4, 4, "N")
)

val employeeDS = employeeDepartment.toDF("employee_id", "department_id", "primary_flag").as[EmployeeDepartment]

import spark.implicits._


defined class EmployeeDepartment
employeeDepartment: Seq[(Int, Int, String)] = List(
  (1, 1, "N"),
  (2, 1, "Y"),
  (2, 2, "N"),
  (3, 3, "N"),
  (4, 2, "N"),
  (4, 3, "Y"),
  (4, 4, "N")
)
employeeDS: Dataset[EmployeeDepartment] = [employee_id: int, department_id: int ... 1 more field]

Employees can belong to multiple departments. When the employee joins other departments, they need to decide which department is their primary department. Note that when an employee belongs to only one department, their primary column is 'N'.

Write a solution to report all the employees with their primary department. For employees who belong to one department, report their only department.

Return the result table in any order.

The result format is in the following example.

| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 1             |
| 3           | 3             |
| 4           | 3             |

In [3]:
employeeDS.groupByKey(row => row.employee_id)
    .mapGroups{
        (employee_id: Int, iter: Iterator[EmployeeDepartment]) => 
        val list = iter.toList
        if(list.length == 1) {
            list.apply(0)
        } else {
            list.filter(x => {x.primary_flag == "Y"}).apply(0)
        }
    }.drop("primary_flag")
    .show()

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            1|
|          3|            3|
|          4|            3|
+-----------+-------------+



In [4]:
val windowSpec = Window.partitionBy(col("employee_id"))
    .orderBy(col("primary_flag").desc)

employeeDS.withColumn("rownum", row_number().over(windowSpec))
    .filter(col("rownum") === 1)
    .drop("primary_flag", "rownum")
    .show()

+-----------+-------------+
|employee_id|department_id|
+-----------+-------------+
|          1|            1|
|          2|            1|
|          3|            3|
|          4|            3|
+-----------+-------------+



windowSpec: WindowSpec = org.apache.spark.sql.expressions.WindowSpec@2f78f3b7

In [5]:
spark.stop()